In [14]:
import re
import heapq 
import copy
import sys
import math
from queue import *

sys.setrecursionlimit(100000)

In [15]:
x1=math.radians(30)

In [16]:
1 - math.sin(math.radians(30))

0.5

In [17]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,0) : (1, 1.5, 1.866),
    (60,0) : (1, 1.866, 1.5),
    (90,0) : (1, 2, 1),
    (120,0) : (1, 1.866, 0.5),
    (150,0) : (1, 1.5, 0.144),
    (180,180) : (1, 1, 0),
    (150,180) : (1, 0.5, 0.144),
    (120,180) : (1, 0.144, 0.5),
    (90,180) : (1, 0, 1),
    (60,180) : (1, 0.144, 1.5),
    (30,180) : (1, 0.5, 1.866),
    
    #(0,0) : (1,1,2)
    (30,270) : (1.5, 1, 1.866),
    (60,270) : (1.866, 1, 1.5),
    (90,270) : (2, 1, 1),
    (120,270) : (1.866, 1, 0.5),
    (150,270) : (1.5, 1, 0.144),
    #(180,180) : (1,1,0)
    (150,90) : (0.5, 1, 0.144),
    (120,90) : (0.144, 1, 0.866),
    (90,90) : (0, 1, 1),
    (60,90) : (0.144, 1, 1.5),
    (30,90) : (0.5, 1, 1.866),
    
    #(90,0) : (1,2,1)
    (90,330) : (1.5, 1.866, 1),
    (90,300) : (1.866, 1.5, 1),
    #(90,270) : (2,1,1)
    (90,240) : (1.866, 0.866, 1),
    (90,210) : (1.5, 0.5, 1),
    #(90,180) : (1,0,1)
    (90,150) : (0.5, 0.5, 1),
    (90,120) : (0.144, 0.866, 1),
    #(90,90) : (0,1,1)
    (90,60) : (0.144, 1.5, 1),
    (90,30) : (0.5, 1.866, 1)
    
}

axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return 2*distance

    

In [18]:
heuristic((90,270), (30,270))

1.9999559995159895

In [69]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("PathN-10.mb")

In [42]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g, path):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        self.path = path
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [71]:
gs = GlobeState(axis0_180, axis90_270, equator, 0, [])
#gs.RCC_axis0_180()

gs.isSolved()

False

In [9]:
gs.getCurrentHeuristic()

35.29746505894831

In [10]:
gs.RC_axis0_180()
gs.getCurrentHeuristic()

32.854022327425035

In [25]:
gs1 = copy.deepcopy(gs)
gs1.equator == gs.equator

True

In [26]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [33]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                q = state
        print (minVal)
        
        #q.g=q.g+(2*(math.pi)/12)
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs1.path.append("RC_axis0_180")
        gs2.RCC_axis0_180()
        gs2.path.append("RCC_axis0_180")
        gs3.RC_axis90_270()
        gs3.path.append("RC_axis90_270")
        gs4.RCC_axis90_270()
        gs4.path.append("RCC_axis90_270")
        gs5.RC_equator()
        gs5.path.append("RC_equator")
        gs6.RCC_equator()
        gs6.path.append("RCC_equator")
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            state.g = q.g + (2*(math.pi)/12)
            f = state.g + state.getCurrentHeuristic()
            
            flag = True 
            
            if(state.g > n):
                flag=False
                
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
                    #print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    if state.g < stateX.g:
                        #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                        openList.remove(stateX)
                        openList.add(state)
                    flag = False
            
            if(flag==True):
                openList.add(state)
        
        #remove q from openList
        print(len(openList))
        openList.remove(q)
        closedList.add(q)
        
        
        

In [10]:
gh = Astar(2000)

5.882910843158052
7
5.897181446855345
11
5.948886450131944
15
5.907866954271529
19
5.999269163502472
23
5.965848952035683
27
5.8318849995747835
31
5.637248906895339
35
5.779546198041928
39
5.882525064283075
43
5.77666390026737
47
5.65492122832322
51
5.789139957601356
55
5.510734260328957
59
5.157511784245024
63
5.119856017937185
67
5.067497425884181
71
5.693979415191131
75
5.528999363604998
79
5.410652322628527
83
5.653689547020497
87
5.616033780712658
91
5.563675188659654
95
5.6974775297477835
99
5.579957314059366
103
5.7036957877912275
107
5.769635692474876
111
5.809397195939527
115
5.82642536939896
119
5.8485589417418895
123
5.8510794415100005
127
5.733559225821583
131
5.932642360418526
135
5.925409098464967
139
5.946905924224978
143
5.98910065972718
147
5.995793184951497
151
5.642570708867564
155
5.795370916131802
159
5.989538266521178
163
5.997277865628779
167
6.017735323489166
171
6.01884432601263
175
5.860805902263094
179
5.820516034092461
183
5.843021716923102
187
5.93005613403

In [13]:
print(str(gh.path))

['RC_axis0_180', 'RCC_equator', 'RC_axis0_180', 'RC_axis0_180', 'RCC_equator', 'RC_axis90_270', 'RCC_equator', 'RC_axis0_180', 'RC_axis0_180', 'RC_equator']


In [52]:
def Astar_Search():
    
    def Astar(n, k, MaxQueLen):
        #initialize open and closed lists
        openList=set()
        closedList=set()

        #add start state to openList
        openList.add(gs)
    
        while(len(openList)!=0):
            #find state with min f val
            minVal=10000000000
            for state in openList:
                if(state.getCurrentHeuristic() + state.g < minVal):
                    minVal = state.getCurrentHeuristic() + state.g
                    q = state
            print (minVal)
        
            #q.g=q.g+(2*(math.pi)/12)
        
            #create successors
            gs1 = copy.deepcopy(q)
            gs2 = copy.deepcopy(q)
            gs3 = copy.deepcopy(q)
            gs4 = copy.deepcopy(q)
            gs5 = copy.deepcopy(q)
            gs6 = copy.deepcopy(q)
        
            gs1.RC_axis0_180()
            gs1.path.append("RC_axis0_180")
            gs2.RCC_axis0_180()
            gs2.path.append("RCC_axis0_180")
            gs3.RC_axis90_270()
            gs3.path.append("RC_axis90_270")
            gs4.RCC_axis90_270()
            gs4.path.append("RCC_axis90_270")
            gs5.RC_equator()
            gs5.path.append("RC_equator")
            gs6.RCC_equator()
            gs6.path.append("RCC_equator")
        
            #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
            #else add it to openList
            for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
                if(state.isSolved()):
                    return (state, k, MaxQueLen)
            
                state.g = q.g + (2*(math.pi)/12)
                f = state.g + state.getCurrentHeuristic()
            
                flag = True 
            
                if(state.g > n):
                    flag=False
                
                for stateX in closedList:
                    if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                        #if stateX.getCurrentHeuristic() + stateX.g < f:
                        flag = False
                        #print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
            
                for stateX in openList:
                    if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                        if state.g < stateX.g:
                            #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
                            openList.remove(stateX)
                            openList.add(state)
                        flag = False
            
                if(flag==True):
                    k=k+1
                    openList.add(state)
        
            #remove q from openList
        
            if(len(openList)>MaxQueLen):
                MaxQueLen = len(openList)
        
            openList.remove(q)
            closedList.add(q)
    
    goal, k, MaxQueLen = Astar(100, 0, 0)
    
    print("Number of states expanded = " + str(k) + "\n")
    print("Max size of Queue = " + str(MaxQueLen) + "\n")
    print("Final Path Length = " + str(len(goal.path)))
    print("Path = " + str(goal.path))
        

In [72]:
Astar_Search()

5.882910843158052
5.897181446855345
5.948886450131944
5.907866954271529
5.999269163502472
5.965848952035683
5.8318849995747835
5.637248906895339
5.779546198041928
5.882525064283075
5.77666390026737
5.65492122832322
5.789139957601356
5.510734260328957
5.157511784245024
5.119856017937185
5.067497425884181
5.693979415191131
5.528999363604998
5.410652322628527
5.653689547020497
5.616033780712658
5.563675188659654
5.6974775297477835
5.579957314059366
5.7036957877912275
5.769635692474876
5.809397195939527
5.82642536939896
5.8485589417418895
5.8510794415100005
5.733559225821583
5.932642360418526
5.925409098464967
5.946905924224978
5.98910065972718
5.995793184951497
5.642570708867564
5.795370916131802
5.989538266521178
5.997277865628779
6.017735323489166
6.01884432601263
5.860805902263094
5.820516034092461
5.843021716923102
5.930056134030762
6.028716162613373
5.906973490669222
6.029274793029946
5.891417033385337
5.76386406798723
5.723574199816596
6.023671698471398
6.035452837122582
6.043847758

In [28]:
cueue = Queue(maxsize=0)

def RBFS(q, fval, cueue):
    if(q.isSolved()):
        return q
    
    gs1 = copy.deepcopy(q)
    gs2 = copy.deepcopy(q)
    gs3 = copy.deepcopy(q)
    gs4 = copy.deepcopy(q)
    gs5 = copy.deepcopy(q)
    gs6 = copy.deepcopy(q)
    
    gs1.RC_axis0_180()
    gs2.RCC_axis0_180()
    gs3.RC_axis90_270()
    gs4.RCC_axis90_270()
    gs5.RC_equator()
    gs6.RCC_equator()
    
    cueue.put(gs1)
    cueue.put(gs2)
    cueue.put(gs3)
    cueue.put(gs4)
    cueue.put(gs5)
    cueue.put(gs6)
    
    minF = 10000000
    
    sentinel = object()
    for state in iter(cueue.get, sentinel):     
        state.g = state.g + (2*(math.pi)/12)
        fval = max(state.g + state.getCurrentHeuristic(), fval)
        if(fval < minF):
            minF = fval
            stateX = state
    print(minF)
    return RBFS(stateX, fval, cueue)

In [ ]:
gh=RBFS(gs, 0, cueue)

In [ ]:
def RBFS_Search():
    cueue = Queue(maxsize=0)
    maxCueLen = 0
    k = 0
    def RBFS(q, fval, cueue):
        if(q.isSolved()):
            return q
        
        k=k+1
    
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
    
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
    
        cueue.put(gs1)
        cueue.put(gs2)
        cueue.put(gs3)
        cueue.put(gs4)
        cueue.put(gs5)
        cueue.put(gs6)
        
        if(cueue.qsize()>maxCueLen):
            maxCueLen = len(cue)
    
        minF = 10000000
    
        sentinel = object()
        for state in iter(cueue.get, sentinel):     
            state.g = state.g + (2*(math.pi)/12)
            fval = max(state.g + state.getCurrentHeuristic(), fval)
            if(fval < minF):
                minF = fval
                stateX = state
        return RBFS(stateX, fval, cueue)
    
    goal=RBFS(gs, 0, cueue)
    
    print("Number of states expanded = " + k + "\n")
    print("Max size of Queue = " + maxCueLen + "\n")
    print("Final Path Length = " + goal.g)

In [ ]:
RBFS_Search()

In [ ]:
cueue = Queue(maxsize=0)
cueue.put(gs)
def BFS():
    while(cueue.empty()!=True):
        q = cueue.get()
        
        if(q.isSolved()):
            return q
        
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
    
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
    
        cueue.put(gs1)
        cueue.put(gs2)
        cueue.put(gs3)
        cueue.put(gs4)
        cueue.put(gs5)
        cueue.put(gs6)

In [ ]:
gh = BFS()

In [ ]:
def BFS_Search():
    cueue = Queue(maxsize=0)
    cueue.put(gs)
    maxCueLen = 0
    k=0
    
    def BFS():
        while(cueue.empty()!=True):
            q = cueue.get()
            
            k=k+1
        
            if(q.isSolved()):
                return q
        
            gs1 = copy.deepcopy(q)
            gs2 = copy.deepcopy(q)
            gs3 = copy.deepcopy(q)
            gs4 = copy.deepcopy(q)
            gs5 = copy.deepcopy(q)
            gs6 = copy.deepcopy(q)
    
            gs1.RC_axis0_180()
            gs2.RCC_axis0_180()
            gs3.RC_axis90_270()
            gs4.RCC_axis90_270()
            gs5.RC_equator()
            gs6.RCC_equator()
    
            cueue.put(gs1)
            cueue.put(gs2)
            cueue.put(gs3)
            cueue.put(gs4)
            cueue.put(gs5)
            cueue.put(gs6)
            
            if(cueue.qsize()>maxCueLen):
            maxCueLen = len(cue)
    
    goal = BFS()
    
    print("Number of states expanded = " + k + "\n")
    print("Max size of Queue = " + maxCueLen + "\n")
    print("Final Path Length = " + goal.g)

In [ ]:

    
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance

    